In [1]:
import pandas as pd
import re
import json 
import ast

In [5]:
r = re.compile(
    "(?P<time>\d{2}:\d{2}:\d{2}\.\d{3}) \[.*\] INFO  ru.otus.l041.gclog.Slf4jGCLogger.log - (?P<data>.*)"
)

with open("log/out.log") as log_file:
    lines = log_file.readlines()
   
gc_info = [
    {**{"time": m.group("time")}, **(json.loads(m.group("data")))}
    for m in map(r.match, lines)
]

In [7]:
gc_df = pd.DataFrame(gc_info)
gc_df.head()

,count,gcName,spentTime,time
0,154,G1 Young Generation,6982,20:52:14.266
1,25,G1 Old Generation,11942,20:52:14.269
2,109,G1 Young Generation,3136,20:53:14.235
3,44,G1 Old Generation,26971,20:53:14.235
4,250,G1 Young Generation,9597,20:54:14.235


In [9]:
gc_groups = {
    "G1 Young Generation": "G1", 
    "G1 Old Generation": "G1",
    "ParNew": "CMS",
    "ConcurrentMarkSweep": "CMS",
    "PS Scavenge": "Parallel",
    "PS MarkSweep": "Parallel",
    "Copy": "Serial",
    "MarkSweepCompact": "Serial"
}

In [15]:
gc_df["gcAlg"] = gc_df["gcName"].apply(gc_groups.get)
gc_df.head()

,count,gcName,spentTime,time,gcAlg
0,154,G1 Young Generation,6982,20:52:14.266,G1
1,25,G1 Old Generation,11942,20:52:14.269,G1
2,109,G1 Young Generation,3136,20:53:14.235,G1
3,44,G1 Old Generation,26971,20:53:14.235,G1
4,250,G1 Young Generation,9597,20:54:14.235,G1


In [20]:
gc_df.groupby("gcAlg")\
  .agg({"count": "sum", "spentTime": "sum"})\
  .rename(columns={"count": "total_count", "spentTime": "total_spent_time [ms]"})

,total_count,total_spent_time [ms]
gcAlg,,
CMS,1199,610334
G1,1825,382690
Parallel,1224,288215
Serial,2573,564174
